# Input Source Codes

In [1]:
import os
import re
import nltk
import pandas as pd
import numpy as np
nltk.download('punkt')

from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\haydariday\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
source_codes = []
filenames = []
LOC = []
n_LOC = []
filenames_without_extension = []


tags_count = {}
function_dict = {}
function_token = {}

stop_condition = ('for', 'if', 'else', 'while','switch')

tags_count['LOC'] = 0
tags_count['SLOC'] = 0
tags_count['O'] = 0
tags_count['C'] = 0
tags_count['M'] = 0
tags_count['N'] = 0


sc_directory = 'Source_codes/python/'
py_directory = 'python.xls'

In [3]:
%%time

filenames = os.listdir(sc_directory)
for name in filenames :
    i = name.replace('.c','')
    filenames_without_extension.append(i)
filenames

Wall time: 0 ns


['audioop.c',
 'cmathmodule.c',
 'fcntlmodule.c',
 'fpectlmodule.c',
 'fpetestmodule.c',
 'grpmodule.c']

In [4]:
%%time
i = 0
for files in filenames :
    function_dict[filenames_without_extension[i]] = {}
    with open(sc_directory+files,'r') as fh :
        source_codes.append([filenames_without_extension[i],fh.readlines()])
    i += 1

Wall time: 2 ms


In [5]:
%%time

for source_code in source_codes :
    i = 0
    for baris in source_code :
        if not (i % 2 == 0) :
            LOC.append(len(baris))
            tags_count['LOC'] += len(baris)
        i+=1
LOC

Wall time: 0 ns


[1612, 426, 603, 303, 186, 194]

In [6]:
tags_count['LOC']

3324

In [7]:
%%time
for code in source_codes :
    lines_update = []
    for i in range(1,len(code[1])+1) :
        lines_update.append([i,code[1][i-1]])
    code[1] = lines_update

Wall time: 2 ms


# Data Pre-Processing 

## Remove whitespace

In [8]:
%%time
for code in source_codes :
    for i in range(len(code[1])) :
        code[1][i][1] = ' '.join(code[1][i][1].split())
        code[1][i][1] = code[1][i][1].replace('\t', '')
        code[1][i][1] = code[1][i][1].replace('\n', '')
        code[1][i][1] = code[1][i][1].replace('\\n\\', '')
        code[1][i][1] = code[1][i][1].replace('\\n', '')
        code[1][i][1] = code[1][i][1].replace('\\', ' \\ ')
        code[1][i][1] = code[1][i][1].replace('/', ' / ')
        code[1][i][1] = code[1][i][1].replace('\'', ' \' ')
        code[1][i][1] = code[1][i][1].replace('"', ' " ')
        code[1][i][1] = re.sub(r"^\s+","",code[1][i][1])

Wall time: 14 ms


## Split Codes

In [9]:
%%time
for source_code in source_codes :
    i = 0
    for baris in source_code[1] :
        if not (baris[1] == '') :
            baris[1] = word_tokenize(baris[1])
        i += 1

Wall time: 259 ms


## Remove Comments

### Tag Comments

In [10]:
%%time
petik = 0
for source_code in source_codes :
    is_comment = False
    is_comment_line = False
    baris_index = 0
    while (baris_index < len(source_code[1])) :
        word_index = 0
        while (word_index < len(source_code[1][baris_index][1])) :
            tag = 'O'
            tags_count['O'] += 1
            #print(source_code[1][baris_index][1][word_index],end=' ')
            if not(is_comment or is_comment_line) : 
                out_of_bound = (len(source_code[1][baris_index][1])-word_index <= 1)
                #print('o',end=' ')
                if not(out_of_bound) :
                    #print('baris :',baris_index)
                    if (source_code[1][baris_index][1][word_index] == '/') and (source_code[1][baris_index][1][word_index+1] == '*') :
                        petik = 0
                        for indexes in range(word_index) :
                            #print(source_code[1][baris_index][1][indexes][0])
                            if (source_code[1][baris_index][1][indexes][0] == "``" or source_code[1][baris_index][1][indexes][0] == "'" or source_code[1][baris_index][1][indexes][0] == '"') :
                                petik += 1
                        print(petik)
                        if (petik == 0 or petik%2 == 0) :
                            is_comment = True
                            tag = 'C'
                            tags_count['O'] -= 1
                        #print('yes1',end=' ')
                    elif (source_code[1][baris_index][1][word_index] == '/') and (source_code[1][baris_index][1][word_index+1] == '/') :
                        is_comment_line = True
                        tag = 'C'
                        tags_count['O'] -= 1
                        #print('yes2',end=' ')
            elif (is_comment or is_comment_line) :
                tag = 'C'
                tags_count['O'] -= 1
                #print('yes3',end=' ')
                if (word_index > 0) and (is_comment) :
                    #print('no1',end=' ')
                    #print(source_code[1][baris_index][1][word_index-1], source_code[1][baris_index][1][word_index])
                    if (source_code[1][baris_index][1][word_index-1][0] == '*') and (source_code[1][baris_index][1][word_index] == '/') :
                        #print('no2',end=' ')
                        if (word_index > 1) :
                            #print('no3',end=' ')
                            if not (source_code[1][baris_index][1][word_index-2][0] == '/') :
                                is_comment = False
                                #print('no',end=' ')
                        else :
                            is_comment = False
            source_code[1][baris_index][1][word_index] = [source_code[1][baris_index][1][word_index], tag]
            word_index += 1
            #print('oh')
        baris_index += 1
        is_comment_line = False

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Wall time: 40 ms


### Removing Comments

In [11]:
%%time

for source_code in source_codes :
    baris_index = 0
    while (baris_index < len(source_code[1])) :
        word_index = 0
        while (word_index < len(source_code[1][baris_index][1])) :
            if (source_code[1][baris_index][1][word_index][1] == 'C') :
                source_code[1][baris_index][1].pop(word_index)
                tags_count['C'] += 1
            else :
                word_index += 1
        baris_index += 1

Wall time: 13 ms


## Remove Empty Baris

In [12]:
%%time

for source_code in source_codes :
    baris_index = 0   
    while (baris_index < len(source_code[1])) :
        if len(source_code[1][baris_index][1]) <= 0 :
            source_code[1].pop(baris_index)
        else :
            baris_index += 1
    n_LOC.append(len(source_code[1]))
    tags_count['SLOC'] += len(source_code[1])

Wall time: 2 ms


In [13]:
n_LOC

[1233, 347, 519, 171, 119, 163]

In [14]:
tags_count['SLOC']

2552

# Tag Function

In [15]:
def check_is_function(source_code, index_word, index_baris, dict_function) :
    found = False #awali dengan kondisi tidak ditemukan
    
    #kondisi ketika pointer '{' merupakan urutan awal dari barisan token
    if (index_word <= 0) :
        try :
            index_baris = index_baris - 1
            index_word = len(source_code[1][index_baris][1]) - 1
        except :
            print("something went wrong, pointer can't get back to previous line")
    
    #kondisi ketika pointer '{' bukan urutan awal dari barisan
    else :
        index_word -=  1
        
    #kondisi ketika tidak menemukan token ')'
    #print('1.5',source_code[1][index_baris][1][index_word][0])
    if (not(source_code[1][index_baris][1][index_word][0] == ')')) :
        #print('why0')
        return False
    
    #cek apakah pointer berada pada baris awal dari source code
    if not(index_baris<=0) :
        #cek apakah pointer ada didalam macro
        if (source_code[1][index_baris-1][1][len(source_code[1][index_baris-1][1])-1][0]=='\\') :
            #print('why1')
            return False
    if (source_code[1][index_baris][1][0][0]=='#') :
        #print('why2')
        return False
    
    while (index_baris >= 0 and not(found)) :
        while (index_word >= 0 and not(found)) :
            #print('2',source_code[1][index_baris][1][index_word][0])
            if (source_code[1][index_baris][1][index_word][0] in stop_condition) :
                #print('why3')
                return False
            elif (source_code[1][index_baris][1][index_word][0] in (';','}')) :
                found = True;
            else :
                index_word -= 1
        if not(found) :
            index_baris = index_baris - 1
            index_word  = len(source_code[1][index_baris][1]) - 1
            
            # cek apakah baris macro
            #print('3',source_code[1][index_baris][1][index_word][0])
            if not(index_baris<=0) :
            #cek apakah pointer ada didalam macro
                #print('4',source_code[1][index_baris-1][1][len(source_code[1][index_baris-1][1])-1][0])
                if (source_code[1][index_baris-1][1][len(source_code[1][index_baris-1][1])-1][0]=='\\') :
                    found = True
            if (source_code[1][index_baris][1][0][0]=='#') :
                found = True

    if (len(source_code[1][index_baris][1]) - index_word <= 1 ) :
        index_baris += 1
        index_word = 0
    else :
        index_word += 1
        
    count = 0
    count_on = False
    is_named = False
    lines = []
    function_name = ''
    file_name = ''
    
    while (index_baris < len(source_code[1]) and (count>0 or not count_on)) :
        index_word = 0
        lines_temp = []
        lines_temp.append(source_code[1][index_baris][0])
        while (index_word < len(source_code[1][index_baris][1]) and (count>0 or not count_on)) :
            #print('g',source_code[1][index_baris][1][index_word])
            #print(index_baris)
            source_code[1][index_baris][1][index_word][1] = 'M'
            tags_count['M'] += 1
            tags_count['O'] -= 1
            lines_temp.append(source_code[1][index_baris][1][index_word][0])
            if (source_code[1][index_baris][1][index_word][0] == '{') :
                #print('yes0')
                count_on = True
                count += 1
            elif (not(is_named) and source_code[1][index_baris][1][index_word][0] == '(') :
                #print('yes1')
                if (index_word <= 0) :
                    #print('yes2')
                    source_code[1][index_baris-1][1][len(source_code[1][index_baris-1][1]) - 1][1] = 'N'
                    tags_count['M'] -= 1
                    tags_count['N'] += 1
                    function_name = source_code[1][index_baris-1][1][len(source_code[1][index_baris-1][1]) - 1][0]
                else :
                    #print('yes3')
                    source_code[1][index_baris][1][index_word-1][1] = 'N'
                    tags_count['M'] -= 1
                    tags_count['N'] += 1
                    function_name = source_code[1][index_baris][1][index_word-1][0]
                file_name = source_code[0]
                dict_function[file_name.lower()][function_name.lower()] = {}
                is_named = True
            elif (source_code[1][index_baris][1][index_word][0] == '}') :
                #print('yes4')
                count -= 1
            if (count>0 or not(count_on)) :
                index_word += 1
            #print(index_word < len(source_code[1][index_baris][1]) and (count>0 or not count_on))
        #print('final')
        lines.append(lines_temp)
        if (count>0 or not(count_on)) :
            index_baris += 1
    dict_function[file_name.lower()][function_name.lower()]['code'] = {}
    for line in lines :
        dict_function[file_name.lower()][function_name.lower()]['code'][line[0]] = []
        index = 1
        while (index<len(line)) :
            dict_function[file_name.lower()][function_name.lower()]['code'][line[0]].append(line[index])
            index += 1
    return True

In [16]:
%%time
jumlah = 0

for source_code in source_codes :
    baris_index = 0
    while (baris_index < len(source_code[1])) :
        word_index = 0
        while (word_index < len(source_code[1][baris_index][1])) :
            #print(baris_index,word_index)
            if not(source_code[1][baris_index][1][word_index][1] in ('M','N')) :    
                if (source_code[1][baris_index][1][word_index][0] == '{') :
                    if (check_is_function(source_code, word_index, baris_index,function_dict)) :
                        jumlah += 1
            word_index += 1
        baris_index += 1
#print('Jumlah :',jumlah)

Wall time: 41 ms


In [17]:
tags_count['M']

14940

In [18]:
tags_count['N']

78

In [19]:
tags_count['O']

4140

In [20]:
tags_count['C']

4189

In [21]:
tags_count['all'] = tags_count['M'] + tags_count['N'] + tags_count['O'] + tags_count['C']
tags_count['all']

23347

# Tokenization

In [22]:
syntax = ('for', 'while', 'case', 'if', 'do', 'else', 'break', 'continue', 'switch', 'goto', 'return')

def give_token(x) : 
    if x == 'for' :
        return 'A'
    elif x == 'break' :
        return 'B'
    elif x == 'case' :
        return 'C'
    elif x == 'do' :
        return 'D'
    elif x == 'else' :
        return 'E'
    elif x == 'continue' :
        return 'F'
    elif x == 'goto' :
        return 'G'
    elif x == 'while' :
        return 'H'
    elif x == 'if' :
        return 'I'
    elif x == 'switch' :
        return 'J'
    elif x == 'return' :
        return 'K'
    else :
        return False

In [23]:
%%time

is_in_comment1 = False
is_in_comment2 = False
is_in_comment3 = False
is_comment = False
dummy = {}
for file in list(function_dict) :
    dummy[file] = {}
    for function in list(function_dict[file]) :
        dummy[file][function] = {}
        dummy[file][function]['token'] = ''
        #print(function)
        for baris in list(function_dict[file][function]['code']) :
            for token in list(function_dict[file][function]['code'][baris]) :
                #print(token)
                if (token == '"' and not(is_in_comment2) and not(is_in_comment3)) :
                    #print('comment1')
                    if not(is_in_comment1) :
                        dummy[file][function]['token'] += 'T'
                        is_comment = not(is_comment)
                    is_in_comment1 = not(is_in_comment1)
                elif (token == "'" and not(is_in_comment1) and not(is_in_comment3)) :
                    #print('comment2')
                    if not(is_in_comment2) : 
                        dummy[file][function]['token'] += 'T'
                        is_comment = not(is_comment)
                    is_in_comment2 = not(is_in_comment2)
                elif ((token == "``" or token == "''") and not(is_in_comment1) and not(is_in_comment2)) :
                    #print('comment3')
                    if not(is_in_comment3) :
                        dummy[file][function]['token'] += 'T'
                        is_comment = not(is_comment)
                    is_in_comment3 = not(is_in_comment3)
                if (not(is_in_comment1 or is_in_comment2 or is_in_comment3 or is_comment)) :
                    if (re.search(r'^-[\w._]+$',token)) :
                        #print('1',re.findall(r'^-[\w._]+$',token)[0])
                        if (re.findall(r'[\w._]+$',token)[0] in syntax) :
                            dummy[file][function]['token'] += '-'
                            if not (give_token(re.findall(r'[\w._]+$',token)[0])) :
                                raise Exception('"{}" not found!'.format(re.findall(r'[\w._]+$',token)[0]))
                            dummy[file][function]['token'] += give_token(re.findall(r'[\w._]+$',token)[0])
                        else :
                            dummy[file][function]['token'] += 'T'
                    elif (re.search(r'^[\w._]+[^\w._]+$',token)) :
                        #print('2',re.findall(r'^[\w._]+[^\w._]+$',token)[0])
                        if (re.findall(r'^[\w._]+',token)[0] in syntax) :
                            if not (give_token(re.findall(r'^[\w._]+',token)[0])) :
                                raise Exception('"{}" not found!'.format(re.findall(r'^[\w._]+',token)[0]))
                            dummy[file][function]['token'] += give_token(re.findall(r'^[\w._]+',token)[0])
                        else :
                            dummy[file][function]['token'] += 'T'
                        dummy[file][function]['token'] += re.findall(r'[^\w._]+$',token)[0]
                    elif (re.search(r'^[^\w._]+[\w._]+$',token)) :
                        #print('3',re.findall(r'^[^\w._]+[\w._]+$',token)[0])
                        dummy[file][function]['token'] += re.findall(r'^[^\w._]+',token)[0]
                        if (re.findall(r'[\w._]+$',token)[0] in syntax) :
                            if not (give_token(re.findall(r'[\w._]+$',token)[0])) :
                                raise Exception('"{}" not found!'.format(re.findall(r'[\w._]+$',token)[0]))
                            dummy[file][function]['token'] += give_token(re.findall(r'[\w._]+$',token)[0])
                        else :
                            dummy[file][function]['token'] += 'T'
                    elif(re.search(r'^[\w.]+[^\w.]+[\w.]+$',token)) :
                        #print('4',re.findall(r'^[\w.]+[^\w.]+[\w.]+$',token)[0])
                        if (re.findall(r'^[\w._]+',token)[0] in syntax) :
                            if not (give_token(re.findall(r'^[\w._]+',token)[0])) :
                                raise Exception('"{}" not found!'.format(re.findall(r'^[\w._]+',token)[0]))
                            dummy[file][function]['token'] += give_token(re.findall(r'^[\w._]+',token)[0])
                        else :
                            dummy[file][function]['token'] += 'T'
                        dummy[file][function]['token'] += re.findall(r'[^\w._]+',token)[0]
                        if (re.findall(r'[\w._]+$',token)[0] in syntax) :
                            if not (give_token(re.findall(r'[\w._]+$',token)[0])) :
                                raise Exception('"{}" not found!'.format(re.findall(r'[\w._]+$',token)[0]))
                            dummy[file][function]['token'] += give_token(re.findall(r'[\w._]+$',token)[0])
                        else :
                            dummy[file][function]['token'] += 'T'
                    elif (re.search(r'^[\w._]+$',token)) :
                        #print('5',re.findall(r'^[\w._]+$',token)[0])
                        if (re.findall(r'^[\w._]+$',token)[0] in syntax) :
                            if not (give_token(re.findall(r'^[\w._]+$',token)[0])) :
                                raise Exception('"{}" not found!'.format(re.findall(r'^[\w._]+$',token)[0]))
                            dummy[file][function]['token'] += give_token(re.findall(r'^[\w._]+$',token)[0])
                        else :
                            #print('why :',re.findall(r'^[\w._]+$',token)[0])
                            dummy[file][function]['token'] += 'T'
                    else :
                        dummy[file][function]['token'] += token
                elif (not(is_in_comment1 or is_in_comment2 or is_in_comment3)) :
                    #print('NO')
                    is_comment = not(is_comment)
                    
for file in list(function_dict) :
    for function in list(function_dict[file]) :
        function_dict[file][function]['token'] = dummy[file][function]['token']

Wall time: 81 ms


In [24]:
i = 0
for file in function_dict :
    for function in function_dict[file] :
        function_token[file+'..'+function+'()']= function_dict[file][function]['token']

# Matching

In [25]:
import textdistance as td

## Compare Functions

In [26]:
for file in function_dict :
    for function in function_dict[file] :
        print(file,function,function_dict[file][function]['token'])

audioop search TTT(TT,T*T,TT){TT;A(T=T;T<T;T++){I(T<=*T++)K(T);}K(T);}
audioop st_14linear2ulaw TTTT(TT){TT;TT;TTT;T=T>>T;I(T<T){T=T;T=T;}E{T=T;}I(T>T)T=T;T+=(T>>T);T=T(T,T,T);I(T>=T)K(TT)(T^T);E{T=(TT)(T<<T)|((T>>(T+T))&T);K(T^T);}}
audioop st_linear2alaw TTTT(TT){TT;TT;TTT;T=T>>T;I(T>=T){T=T;}E{T=T;T=T-T;}T=T(T,T,T);I(T>=T)K(TT)(T^T);E{T=(TT)T<<T;I(T<T)T|=(T>>T)&T;ET|=(T>>T)&T;K(T^T);}}
audioop audioop_getsample TT*T(T*T,T*T){TT*T;TT,T,T=T;TT;I(!T(T,T,&T,&T,&T,&T))KT;I(T!=T&&T!=T&&T!=T){T(T,T);KT;}I(T<T||T>=T/T){T(T,T);KT;}I(T==T)T=(T)*T(T,T);EI(T==T)T=(T)*T(T,T*T);EI(T==T)T=(T)*T(T,T*T);KT(T);}
audioop audioop_max TT*T(T*T,T*T){TT*T;TT,T,T=T;TT;TT=T;I(!T(T,T,&T,&T,&T))KT;I(T!=T&&T!=T&&T!=T){T(T,T);KT;}A(T=T;T<T;T+=T){I(T==T)T=(T)*T(T,T);EI(T==T)T=(T)*T(T,T);EI(T==T)T=(T)*T(T,T);I(T<T)T=(T);I(T>T)T=T;}KT(T);}
audioop audioop_minmax TT*T(T*T,T*T){TT*T;TT,T,T=T;TT;TT=T,T=T;I(!T(T,T,&T,&T,&T))KT;I(T!=T&&T!=T&&T!=T){T(T,T);KT;}A(T=T;T<T;T+=T){I(T==T)T=(T)*T(T,T);EI(T==T)T=(T)*T(T,T);EI(T

### ratcliff_obershelp

In [27]:
%%time
i = 0
key = list(function_token)
dummy = {}
is_clone10 = []
is_clone20 = []
is_clone30 = []
is_clone40 = []
is_clone50 = []
is_clone60 = []
is_clone70 = []
is_clone80 = []
is_clone90 = []
is_clone100 = []
while i < len(function_token)-1 :
    j = i+1
    while j < len(function_token) :
        if (key[i]<=key[j]) :
            func_to_compare = key[i]+'-'+key[j]
        else :
            func_to_compare = key[j]+'-'+key[i]
        dummy[func_to_compare] = td.ratcliff_obershelp(function_token[key[i]],function_token[key[j]])
        is_clone10.append(True)
        is_clone20.append(True)
        is_clone30.append(True)
        is_clone40.append(True)
        is_clone50.append(True)
        is_clone60.append(True)
        is_clone70.append(True)
        is_clone80.append(True)
        is_clone90.append(True)
        is_clone100.append(True)
        if (dummy[func_to_compare] < 1) :
            is_clone100[-1] = False
        if (dummy[func_to_compare] < 0.9) :
            is_clone90[-1] = False
        if (dummy[func_to_compare] < 0.8) :
            is_clone80[-1] = False
        if (dummy[func_to_compare] < 0.7) :
            is_clone70[-1] = False
        if (dummy[func_to_compare] < 0.6) :
            is_clone60[-1] = False
        if (dummy[func_to_compare] < 0.5) :
            is_clone50[-1] = False    
        if (dummy[func_to_compare] < 0.4) :
            is_clone40[-1] = False
        if (dummy[func_to_compare] < 0.3) :
            is_clone30[-1] = False
        if (dummy[func_to_compare] < 0.2) :
            is_clone20[-1] = False
        if (dummy[func_to_compare] < 0.1) :
            is_clone10[-1] = False
        j += 1
    i += 1

Wall time: 1min 19s


# Format Output

In [28]:

df = pd.DataFrame(data=dummy.items(),columns=['Method','Ratio'])
df['>10%'] = pd.DataFrame(is_clone10)
df['>20%'] = pd.DataFrame(is_clone20)
df['>30%'] = pd.DataFrame(is_clone30)
df['>40%'] = pd.DataFrame(is_clone40)
df['>50%'] = pd.DataFrame(is_clone50)
df['>60%'] = pd.DataFrame(is_clone60)
df['>70%'] = pd.DataFrame(is_clone70)
df['>80%'] = pd.DataFrame(is_clone80)
df['>90%'] = pd.DataFrame(is_clone90)
df['>100%'] = pd.DataFrame(is_clone100)

#df.set_index(['Method'], inplace=True)
df

,Method,Ratio,>10%,>20%,>30%,>40%,>50%,>60%,>70%,>80%,>90%,>100%
0,audioop..search()-audioop..st_14linear2ulaw(),0.406250,True,True,True,True,False,False,False,False,False,False
1,audioop..search()-audioop..st_linear2alaw(),0.359788,True,True,True,False,False,False,False,False,False,False
2,audioop..audioop_getsample()-audioop..search(),0.323651,True,True,True,False,False,False,False,False,False,False
3,audioop..audioop_max()-audioop..search(),0.357143,True,True,True,False,False,False,False,False,False,False
4,audioop..audioop_minmax()-audioop..search(),0.348837,True,True,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
2998,grpmodule..grp_getgrall()-grpmodule..grp_getgr...,0.551724,True,True,True,True,True,False,False,False,False,False
2999,grpmodule..grp_getgrgid()-grpmodule..initgrp(),0.518519,True,True,True,True,True,False,False,False,False,False
3000,grpmodule..grp_getgrall()-grpmodule..grp_getgr...,0.615385,True,True,True,True,True,True,False,False,False,False
3001,grpmodule..grp_getgrnam()-grpmodule..initgrp(),0.550898,True,True,True,True,True,False,False,False,False,False


In [29]:
df.sort_values(['Ratio'], ascending=False, axis=0, inplace=True)
df

,Method,Ratio,>10%,>20%,>30%,>40%,>50%,>60%,>70%,>80%,>90%,>100%
1626,audioop..audioop_alaw2lin()-audioop..audioop_u...,1.000000,True,True,True,True,True,True,True,True,True,True
1969,cmathmodule..c_acosh()-cmathmodule..c_asinh(),1.000000,True,True,True,True,True,True,True,True,True,True
2228,cmathmodule..c_cosh()-cmathmodule..c_sinh(),1.000000,True,True,True,True,True,True,True,True,True,True
1573,audioop..audioop_lin2alaw()-audioop..audioop_l...,1.000000,True,True,True,True,True,True,True,True,True,True
2189,cmathmodule..c_cos()-cmathmodule..c_sinh(),1.000000,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2831,fcntlmodule..all_ins()-grpmodule..initgrp(),0.046125,False,False,False,False,False,False,False,False,False,False
57,audioop..search()-fcntlmodule..all_ins(),0.045028,False,False,False,False,False,False,False,False,False,False
1536,audioop..audioop_ratecv()-cmathmodule..c_prodi(),0.042831,False,False,False,False,False,False,False,False,False,False
2388,cmathmodule..c_prodi()-fcntlmodule..all_ins(),0.040619,False,False,False,False,False,False,False,False,False,False


In [30]:
df['Method']=df['Method'].str.lower()
df

,Method,Ratio,>10%,>20%,>30%,>40%,>50%,>60%,>70%,>80%,>90%,>100%
1626,audioop..audioop_alaw2lin()-audioop..audioop_u...,1.000000,True,True,True,True,True,True,True,True,True,True
1969,cmathmodule..c_acosh()-cmathmodule..c_asinh(),1.000000,True,True,True,True,True,True,True,True,True,True
2228,cmathmodule..c_cosh()-cmathmodule..c_sinh(),1.000000,True,True,True,True,True,True,True,True,True,True
1573,audioop..audioop_lin2alaw()-audioop..audioop_l...,1.000000,True,True,True,True,True,True,True,True,True,True
2189,cmathmodule..c_cos()-cmathmodule..c_sinh(),1.000000,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2831,fcntlmodule..all_ins()-grpmodule..initgrp(),0.046125,False,False,False,False,False,False,False,False,False,False
57,audioop..search()-fcntlmodule..all_ins(),0.045028,False,False,False,False,False,False,False,False,False,False
1536,audioop..audioop_ratecv()-cmathmodule..c_prodi(),0.042831,False,False,False,False,False,False,False,False,False,False
2388,cmathmodule..c_prodi()-fcntlmodule..all_ins(),0.040619,False,False,False,False,False,False,False,False,False,False


# Comparison

In [31]:
df_py = pd.read_excel(py_directory)
df_py

,Unnamed: 0,Unnamed: 1,Student,Unnamed: 3,Expert,Nicad-30,Nicad-40,Nicad-50,MeCC_95_20,MeCC_95_10,...,MeCC_70_20,MeCC_70_10,MeCC_70_4,MeCC_60_20,MeCC_60_10,MeCC_60_4,MeCC_50_20,MeCC_50_10,MeCC_50_4,Simcad
0,grpmodule..grp_getgrgid()-audioop..audioop_lin...,NaN,0.0,NaN,No Clone,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,cmathmodule..c_prodi()-audioop..audioop_lin2lin(),NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,cmathmodule..c_prodi()-grpmodule..grp_getgrgid(),NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,fcntlmodule..fcntl_fcntl()-audioop..audioop_li...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
4,fcntlmodule..fcntl_fcntl()-grpmodule..grp_getg...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3008,NaN,NaN,NaN,Type 1,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3009,NaN,NaN,NaN,Type 2,7,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3010,NaN,NaN,NaN,Type 3,6,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3011,NaN,NaN,NaN,Type 4,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df_py = df_py.dropna(subset=['Expert', 'Student'])
df_py = df_py.rename(columns={'Unnamed: 0': 'Method'})
df_py

,Method,Unnamed: 1,Student,Unnamed: 3,Expert,Nicad-30,Nicad-40,Nicad-50,MeCC_95_20,MeCC_95_10,...,MeCC_70_20,MeCC_70_10,MeCC_70_4,MeCC_60_20,MeCC_60_10,MeCC_60_4,MeCC_50_20,MeCC_50_10,MeCC_50_4,Simcad
0,grpmodule..grp_getgrgid()-audioop..audioop_lin...,NaN,0.00,NaN,No Clone,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,fcntlmodule..fcntl_fcntl()-cmathmodule..c_prodi(),NaN,0.00,NaN,No Clone,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,cmathmodule..c_asin()-audioop..audioop_lin2lin(),NaN,0.00,NaN,No Clone,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,fpetestmodule..db0()-audioop..audioop_lin2lin(),NaN,0.00,NaN,No Clone,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,audioop..audioop_cross()-fcntlmodule..ins(),NaN,0.75,NaN,No Clone,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2989,fpetestmodule..nest3()-audioop..initaudioop(),NaN,0.00,NaN,No Clone,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2993,fpetestmodule..nest3()-fpetestmodule..printerr(),NaN,0.00,NaN,No Clone,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2997,fpetestmodule..nest3()-grpmodule..mkgrent(),NaN,0.00,NaN,No Clone,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3001,fpetestmodule..nest3()-audioop..audioop_lin2ad...,NaN,0.50,NaN,No Clone,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
df_py['Expert'].unique()

array(['No Clone', 'Type 3', 'Type 4', 'Type 2', 'Unsure', 'type 2', 15],
      dtype=object)

In [34]:
df_py = df_py[df_py.Expert != 'Unsure']
df_py = df_py[df_py.Expert != 'Not Classified']
df_py = df_py[df_py.Expert != 15]
df_py = df_py[df_py.Expert != 18]
df_py

,Method,Unnamed: 1,Student,Unnamed: 3,Expert,Nicad-30,Nicad-40,Nicad-50,MeCC_95_20,MeCC_95_10,...,MeCC_70_20,MeCC_70_10,MeCC_70_4,MeCC_60_20,MeCC_60_10,MeCC_60_4,MeCC_50_20,MeCC_50_10,MeCC_50_4,Simcad
0,grpmodule..grp_getgrgid()-audioop..audioop_lin...,NaN,0.00,NaN,No Clone,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,fcntlmodule..fcntl_fcntl()-cmathmodule..c_prodi(),NaN,0.00,NaN,No Clone,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,cmathmodule..c_asin()-audioop..audioop_lin2lin(),NaN,0.00,NaN,No Clone,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,fpetestmodule..db0()-audioop..audioop_lin2lin(),NaN,0.00,NaN,No Clone,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,audioop..audioop_cross()-fcntlmodule..ins(),NaN,0.75,NaN,No Clone,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2981,fpetestmodule..nest3()-fpectlmodule..initfpectl(),NaN,0.00,NaN,No Clone,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2989,fpetestmodule..nest3()-audioop..initaudioop(),NaN,0.00,NaN,No Clone,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2993,fpetestmodule..nest3()-fpetestmodule..printerr(),NaN,0.00,NaN,No Clone,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2997,fpetestmodule..nest3()-grpmodule..mkgrent(),NaN,0.00,NaN,No Clone,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
df_py = df_py.rename(columns={'Unnamed: 0':'Method'})

In [36]:
df_lain = df_py.drop(['Unnamed: 1','Unnamed: 3'],axis=1)
df_lain.to_excel('df_lain.xlsx')

In [37]:
df.to_excel('df_py.xlsx')

In [38]:
df_py = df_py[['Method','Expert']]
df_py

,Method,Expert
0,grpmodule..grp_getgrgid()-audioop..audioop_lin...,No Clone
5,fcntlmodule..fcntl_fcntl()-cmathmodule..c_prodi(),No Clone
10,cmathmodule..c_asin()-audioop..audioop_lin2lin(),No Clone
15,fpetestmodule..db0()-audioop..audioop_lin2lin(),No Clone
25,audioop..audioop_cross()-fcntlmodule..ins(),No Clone
...,...,...
2981,fpetestmodule..nest3()-fpectlmodule..initfpectl(),No Clone
2989,fpetestmodule..nest3()-audioop..initaudioop(),No Clone
2993,fpetestmodule..nest3()-fpetestmodule..printerr(),No Clone
2997,fpetestmodule..nest3()-grpmodule..mkgrent(),No Clone


### Check if both result detected same methods

In [39]:
arr_comparable_methods = list(df_py['Method'])

arr_temp = []
for comparable_methods in arr_comparable_methods :
    temp = sorted(comparable_methods.split('-'))
    arr_temp.append(temp)
arr_comparable_methods = arr_temp

In [44]:
list0 = []
for row,col in df.iterrows() :
    methods = df.at[row,'Method'].split('-')
    for method in methods :
        #print(method)
        if (method.lower() in list0) :
            print('no')
        else :
            list0.append(method.lower())
            print('yes')

yes
yes
yes
yes
yes
yes
yes
yes
yes
no
no
yes
yes
yes
yes
yes
no
no
no
no
yes
yes
no
no
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
no
no
no
yes
yes
no
yes
yes
no
no
no
no
no
yes
yes
yes
yes
no
no
no
yes
no
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
yes
yes
no
no
yes
no
yes
no
no
no
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
no
no
no
yes
yes
no
no
no
no
no
no
yes
yes
yes
yes
no
no
no
no
no
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
yes
yes
yes
yes
no
no
no
yes
yes
no
no
no
no
no
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
no
no
no
yes
no
no
yes
no
no
yes
no
no
no
no
no
no
no
no
no
yes
yes
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
yes
yes
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no

In [45]:
list1 = []
for comparable_methods in arr_comparable_methods :
    for method in comparable_methods :
        #print(method)
        if (method.lower() in list1) :
            print('no')
        else :
            list1.append(method.lower())
            print('yes')

yes
yes
yes
yes
no
yes
no
yes
yes
yes
yes
no
no
no
no
yes
no
no
no
yes
no
no
no
yes
yes
no
no
yes
no
no
yes
no
no
no
yes
no
no
no
no
yes
no
no
yes
no
no
no
no
no
no
yes
no
no
no
no
no
yes
no
no
no
yes
no
no
no
no
no
yes
no
no
no
no
no
no
yes
no
no
no
no
no
no
yes
no
no
no
no
no
no
yes
no
no
no
no
no
yes
no
no
no
no
no
no
yes
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
yes
no
no
no
no
no
no
yes
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
yes


#### Check if there are any differences

In [46]:
set(list1).issubset(set(list0))

True

In [47]:
set(list0).difference(set(list1))

set()

#### Get all the comparable method

In [48]:
for row in df_py['Method'] :
    key=row.split('-')
    if (key[0]<=key[1]) :
        new_key = key[0]+'-'+key[1]
    else :        
        new_key = key[1]+'-'+key[0]
        #print(new_key+" df_py")
    df_py=df_py.replace(to_replace=row, value=new_key)
    
for row in df['Method'] :
    key=row.split('-')
    if (key[0]<=key[1]) :
        new_key = key[0]+'-'+key[1]
    else :
        new_key = key[1]+'-'+key[0]
        #print(new_key+" df")
    df=df.replace(to_replace=row, value=new_key)

In [49]:
df_py=df_py.set_index('Method')
df_py

,Expert
Method,
audioop..audioop_lin2lin()-grpmodule..grp_getgrgid(),No Clone
cmathmodule..c_prodi()-fcntlmodule..fcntl_fcntl(),No Clone
audioop..audioop_lin2lin()-cmathmodule..c_asin(),No Clone
audioop..audioop_lin2lin()-fpetestmodule..db0(),No Clone
audioop..audioop_cross()-fcntlmodule..ins(),No Clone
...,...
fpectlmodule..initfpectl()-fpetestmodule..nest3(),No Clone
audioop..initaudioop()-fpetestmodule..nest3(),No Clone
fpetestmodule..nest3()-fpetestmodule..printerr(),No Clone


In [50]:
df.to_excel('df_py.xlsx')

In [51]:
df = df.set_index('Method')
df

,Ratio,>10%,>20%,>30%,>40%,>50%,>60%,>70%,>80%,>90%,>100%
Method,,,,,,,,,,,
audioop..audioop_alaw2lin()-audioop..audioop_ulaw2lin(),1.000000,True,True,True,True,True,True,True,True,True,True
cmathmodule..c_acosh()-cmathmodule..c_asinh(),1.000000,True,True,True,True,True,True,True,True,True,True
cmathmodule..c_cosh()-cmathmodule..c_sinh(),1.000000,True,True,True,True,True,True,True,True,True,True
audioop..audioop_lin2alaw()-audioop..audioop_lin2ulaw(),1.000000,True,True,True,True,True,True,True,True,True,True
cmathmodule..c_cos()-cmathmodule..c_sinh(),1.000000,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...
fcntlmodule..all_ins()-grpmodule..initgrp(),0.046125,False,False,False,False,False,False,False,False,False,False
audioop..search()-fcntlmodule..all_ins(),0.045028,False,False,False,False,False,False,False,False,False,False
audioop..audioop_ratecv()-cmathmodule..c_prodi(),0.042831,False,False,False,False,False,False,False,False,False,False


In [52]:
df.to_excel('df.xlsx')

In [53]:
df_final=pd.concat([df, df_py], axis=1)
df_final = df_final.dropna(subset=['Expert'])
df_final['Expert'] = df_final['Expert'].replace(to_replace=r'^No Clone$', value= False, regex=True)
df_final['Expert'] = df_final['Expert'].replace(to_replace=r'^Type.*$', value= True, regex=True)
df_final['Expert'] = df_final['Expert'].replace(to_replace=r'^type.*$', value= True, regex=True)

In [54]:
df_final

,Ratio,>10%,>20%,>30%,>40%,>50%,>60%,>70%,>80%,>90%,>100%,Expert
audioop..audioop_avgpp()-audioop..audioop_maxpp(),0.949939,True,True,True,True,True,True,True,True,True,False,True
audioop..audioop_avg()-audioop..audioop_minmax(),0.920097,True,True,True,True,True,True,True,True,True,False,True
audioop..audioop_max()-audioop..audioop_rms(),0.908654,True,True,True,True,True,True,True,True,True,False,True
audioop..audioop_avg()-audioop..audioop_max(),0.904177,True,True,True,True,True,True,True,True,True,False,True
audioop..audioop_mul()-audioop..audioop_tomono(),0.878866,True,True,True,True,True,True,True,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
audioop..audioop_ratecv()-cmathmodule..c_sinh(),0.071560,False,False,False,False,False,False,False,False,False,False,False
cmathmodule..c_acos()-fcntlmodule..all_ins(),0.070142,False,False,False,False,False,False,False,False,False,False,False
cmathmodule..math_error()-fcntlmodule..all_ins(),0.069943,False,False,False,False,False,False,False,False,False,False,False
audioop..audioop_ratecv()-fcntlmodule..conv_descriptor(),0.067890,False,False,False,False,False,False,False,False,False,False,False


In [55]:
for ratio in ['>10%','>20%','>30%','>40%','>50%','>60%','>70%','>80%','>90%','>100%'] :
    comparison_column = np.where((df_final[ratio] == df_final['Expert']) , True, False)
    df_final[ratio+' Result'] = comparison_column

In [56]:
df_final

,Ratio,>10%,>20%,>30%,>40%,>50%,>60%,>70%,>80%,>90%,...,>10% Result,>20% Result,>30% Result,>40% Result,>50% Result,>60% Result,>70% Result,>80% Result,>90% Result,>100% Result
audioop..audioop_avgpp()-audioop..audioop_maxpp(),0.949939,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,False
audioop..audioop_avg()-audioop..audioop_minmax(),0.920097,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,False
audioop..audioop_max()-audioop..audioop_rms(),0.908654,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,False
audioop..audioop_avg()-audioop..audioop_max(),0.904177,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,False
audioop..audioop_mul()-audioop..audioop_tomono(),0.878866,True,True,True,True,True,True,True,True,False,...,True,True,True,True,True,True,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
audioop..audioop_ratecv()-cmathmodule..c_sinh(),0.071560,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
cmathmodule..c_acos()-fcntlmodule..all_ins(),0.070142,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
cmathmodule..math_error()-fcntlmodule..all_ins(),0.069943,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
audioop..audioop_ratecv()-fcntlmodule..conv_descriptor(),0.067890,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True


In [57]:
df_final.to_excel("output.xlsx") 

In [58]:
confusion_matrix = pd.crosstab(df_final['Expert'], df_final['>10%'], rownames=['Actual'], colnames=['Predicted'])
print(">10% Accuracy :",accuracy_score(df_final['Expert'],df_final['>10%']))
print(">10% Precision :",precision_score(df_final['Expert'],df_final['>10%']))
print(">10% Recall :",recall_score(df_final['Expert'],df_final['>10%']))
print(">10% F1 :",f1_score(df_final['Expert'],df_final['>10%']))
confusion_matrix

>10% Accuracy : 0.05504587155963303
>10% Precision : 0.02830188679245283
>10% Recall : 1.0
>10% F1 : 0.055045871559633024


Predicted,False,True
Actual,,
False,15,515
True,0,15


In [59]:
confusion_matrix = pd.crosstab(df_final['Expert'], df_final['>20%'], rownames=['Actual'], colnames=['Predicted'])
print(">20% Accuracy :",accuracy_score(df_final['Expert'],df_final['>20%']))
print(">20% Precision :",precision_score(df_final['Expert'],df_final['>20%']))
print(">20% Recall :",recall_score(df_final['Expert'],df_final['>20%']))
print(">20% F1 :",f1_score(df_final['Expert'],df_final['>20%']))
confusion_matrix

>20% Accuracy : 0.24036697247706423
>20% Precision : 0.03496503496503497
>20% Recall : 1.0
>20% F1 : 0.06756756756756757


Predicted,False,True
Actual,,
False,116,414
True,0,15


In [60]:
confusion_matrix = pd.crosstab(df_final['Expert'], df_final['>30%'], rownames=['Actual'], colnames=['Predicted'])
print(">30% Accuracy :",accuracy_score(df_final['Expert'],df_final['>30%']))
print(">30% Precision :",precision_score(df_final['Expert'],df_final['>30%']))
print(">30% Recall :",recall_score(df_final['Expert'],df_final['>30%']))
print(">30% F1 :",f1_score(df_final['Expert'],df_final['>30%']))
confusion_matrix

>30% Accuracy : 0.43853211009174314
>30% Precision : 0.04672897196261682
>30% Recall : 1.0
>30% F1 : 0.08928571428571429


Predicted,False,True
Actual,,
False,224,306
True,0,15


In [61]:
confusion_matrix = pd.crosstab(df_final['Expert'], df_final['>40%'], rownames=['Actual'], colnames=['Predicted'])
print(">40% Accuracy :",accuracy_score(df_final['Expert'],df_final['>40%']))
print(">40% Precision :",precision_score(df_final['Expert'],df_final['>40%']))
print(">40% Recall :",recall_score(df_final['Expert'],df_final['>40%']))
print(">40% F1 :",f1_score(df_final['Expert'],df_final['>40%']))
confusion_matrix

>40% Accuracy : 0.6513761467889908
>40% Precision : 0.07317073170731707
>40% Recall : 1.0
>40% F1 : 0.13636363636363635


Predicted,False,True
Actual,,
False,340,190
True,0,15


In [62]:
confusion_matrix = pd.crosstab(df_final['Expert'], df_final['>50%'], rownames=['Actual'], colnames=['Predicted'])
print(">50% Accuracy :",accuracy_score(df_final['Expert'],df_final['>50%']))
print(">50% Precision :",precision_score(df_final['Expert'],df_final['>50%']))
print(">50% Recall :",recall_score(df_final['Expert'],df_final['>50%']))
print(">50% F1 :",f1_score(df_final['Expert'],df_final['>50%']))
confusion_matrix

>50% Accuracy : 0.8091743119266055
>50% Precision : 0.12605042016806722
>50% Recall : 1.0
>50% F1 : 0.22388059701492538


Predicted,False,True
Actual,,
False,426,104
True,0,15


In [63]:
confusion_matrix = pd.crosstab(df_final['Expert'], df_final['>60%'], rownames=['Actual'], colnames=['Predicted'])
print(">60% Accuracy :",accuracy_score(df_final['Expert'],df_final['>60%']))
print(">60% Precision :",precision_score(df_final['Expert'],df_final['>60%']))
print(">60% Recall :",recall_score(df_final['Expert'],df_final['>60%']))
print(">60% F1 :",f1_score(df_final['Expert'],df_final['>60%']))
confusion_matrix

>60% Accuracy : 0.9119266055045872
>60% Precision : 0.23809523809523808
>60% Recall : 1.0
>60% F1 : 0.3846153846153846


Predicted,False,True
Actual,,
False,482,48
True,0,15


In [64]:
confusion_matrix = pd.crosstab(df_final['Expert'], df_final['>70%'], rownames=['Actual'], colnames=['Predicted'])
print(">70% Accuracy :",accuracy_score(df_final['Expert'],df_final['>70%']))
print(">70% Precision :",precision_score(df_final['Expert'],df_final['>70%']))
print(">70% Recall :",recall_score(df_final['Expert'],df_final['>70%']))
print(">70% F1 :",f1_score(df_final['Expert'],df_final['>70%']))
confusion_matrix

>70% Accuracy : 0.9761467889908257
>70% Precision : 0.5357142857142857
>70% Recall : 1.0
>70% F1 : 0.6976744186046512


Predicted,False,True
Actual,,
False,517,13
True,0,15


In [65]:
confusion_matrix = pd.crosstab(df_final['Expert'], df_final['>80%'], rownames=['Actual'], colnames=['Predicted'])
print(">80% Accuracy :",accuracy_score(df_final['Expert'],df_final['>80%']))
print(">80% Precision :",precision_score(df_final['Expert'],df_final['>80%']))
print(">80% Recall :",recall_score(df_final['Expert'],df_final['>80%']))
print(">80% F1 :",f1_score(df_final['Expert'],df_final['>80%']))
confusion_matrix

>80% Accuracy : 0.9944954128440368
>80% Precision : 0.9285714285714286
>80% Recall : 0.8666666666666667
>80% F1 : 0.896551724137931


Predicted,False,True
Actual,,
False,529,1
True,2,13


In [66]:
confusion_matrix = pd.crosstab(df_final['Expert'], df_final['>90%'], rownames=['Actual'], colnames=['Predicted'])
print(">90% Accuracy :",accuracy_score(df_final['Expert'],df_final['>90%']))
print(">90% Precision :",precision_score(df_final['Expert'],df_final['>90%']))
print(">90% Recall :",recall_score(df_final['Expert'],df_final['>90%']))
print(">90% F1 :",f1_score(df_final['Expert'],df_final['>90%']))
confusion_matrix

>90% Accuracy : 0.9798165137614679
>90% Precision : 1.0
>90% Recall : 0.26666666666666666
>90% F1 : 0.4210526315789474


Predicted,False,True
Actual,,
False,530,0
True,11,4


In [67]:
confusion_matrix = pd.crosstab(df_final['Expert'], df_final['>100%'], rownames=['Actual'], colnames=['Predicted'])
print(">100% Accuracy :",accuracy_score(df_final['Expert'],df_final['>100%']))
print(">100% Precision :",precision_score(df_final['Expert'],df_final['>100%']))
print(">100% Recall :",recall_score(df_final['Expert'],df_final['>100%']))
print(">100% F1 :",f1_score(df_final['Expert'],df_final['>100%']))
confusion_matrix

>100% Accuracy : 0.9724770642201835
>100% Precision : 0.0
>100% Recall : 0.0
>100% F1 : 0.0


C:\Users\haydariday\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Predicted,False
Actual,
False,530
True,15
